In [1]:
import requests
import re
from bs4 import BeautifulSoup
import time
import json
import numpy as np
import pandas as pd

In [25]:
def get_data(product_link):    
    response = requests.get(product_link)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    product_id = re.findall(R'P[0-9]{3,6}', product_link)[0]
    
    if soup.find('h1', class_='css-56434t'):
        return None

    # Get brand name and product name
    for brand_and_name in soup.find_all('h1', class_='css-a1jw00'):
        names = [names.get_text() for names in brand_and_name.find_all('span')]
        brand = names[0]
        prd_name = names[1]

    # Get Category and price (use json to get categories and price),
    # Because it would give us sub categories and offer price.
    # But if that doesn't work, just get main category and price box.
    dic_json = soup.find_all(attrs={"type": "application/ld+json"})
    try:
        json_category = json.loads(dic_json[0].get_text())
        price = json.loads(dic_json[1].get_text())['offers'][0]['price']
        category_lst = [item['item']['name']
                        for item in json_category['itemListElement']]
        category = '/'.join(category_lst)

    except:
        try:
            category = [cate.get_text() for cate in soup.find_all('ol', class_='css-1doqpel')]
            price = soup.find('div', attrs={"data-comp": "Price Box"}).get_text()
        except:
            price = 'na'
            category = 'na'

    # Get item number and size
    size_and_item = soup.find(attrs={"data-comp": "SizeAndItemNumber Box"})
    # If there's only item number and no size info, it will be out of index
    try:
        item_num = size_and_item.contents[1].split(' ')[1]
    except:
        item_num = size_and_item.contents[0].split(' ')[1]
    # If there's no size at SizeAndItemNumber Box, then try use description area to see.
    try:
        size = size_and_item.span.contents[0].split('SIZE ')[1]
    except:
        try:
            size = soup.find(
                "span", attrs={"class": "css-12wl10d"}).contents[-1]
        except:
            size = 'na'

    # Get love counts
    try:
        love_counts = soup.find(
            'span', attrs={"data-at": "product_love_count"}).get_text()
    except:
        love_counts = 'na'

    # review nums and ratings
    link_json = soup.find(attrs={"id": "linkJSON"})
    json_str = str(link_json)
    ratings = re.findall(R'\"rating\"\:(.*?)\,', json_str)
    reviews = re.findall(R'\"reviews\"\:(.*?)\,', json_str)
    try:
        rating = ratings[0]
    except:
        rating = 'na'
    try:
        reviews_count = reviews[0]
    except:
        reviews_count = 'na'

    dic1 = {}
    dic1['category'] = category
    dic1['product_name'] = prd_name
    dic1['brand'] = brand
    dic1['price'] = price
    dic1['Product_Id'] = product_id
    dic1['size'] = size
    dic1['item_num'] = item_num
    dic1['love_count'] = love_counts
    dic1['rating'] = rating
    dic1['reviews_count'] = reviews_count
    dic1['link'] = product_link
    return dic1

In [3]:
# Test: get first 5 product info
products = open('product_link.txt')
i = 0
frame = []
for item in products:
    link = item.rstrip()
    product_data = get_data(link)
    if product_data:
        df = pd.DataFrame(product_data,index=[i])
        frame.append(df)
        i+=1
        time.sleep(np.random.random()*3)
    if i == 5:
        break
result = pd.concat(frame)
result

,category,product_name,brand,price,Product_Id,size,item_num,love_count,rating,reviews_count,link
0,Fragrance/Value & Gift Sets/Perfume Gift Sets,Blu Mediterraneo MINIATURE Set,Acqua Di Parma,63.0,P443401,5 x 0.16oz/5mL,2218774,2594,4,4,https://www.sephora.com/product/blu-mediterran...
1,Fragrance/Men/Cologne,Colonia,Acqua Di Parma,63.0,P163604,0.7 oz/ 20 mL,2044816,2528,4.3067,75,https://www.sephora.com/product/colonia-P16360...
2,Fragrance/Women/Perfume,Mirto di Panarea,Acqua Di Parma,64.0,P307804,2.5 oz/ 74 mL,1417617,2621,4.6364,22,https://www.sephora.com/product/blu-mediterran...
3,Fragrance/Women/Perfume,Rosa Nobile,Acqua Di Parma,68.0,P388670,3.4 oz/ 101 mL,1638832,4574,4.5652,69,https://www.sephora.com/product/rosa-nobile-P3...
4,Fragrance/Value & Gift Sets/Perfume Gift Sets,Colonia Miniature Set,Acqua Di Parma,69.0,P443400,5 x 0.16oz/5mL,2218766,778,3.5,2,https://www.sephora.com/product/colonia-miniat...


In [5]:
## How many products? 
products = open('product_link.txt')
count = 0
for item in products:
    count += 1
print('Product counts: ', count)

Product counts:  3093


### There are over 3000 products on Sephora website. 
### But It seems that Sephora has set limits for request times each IP address and each day. 
### Therefore, we can try to scape all the product information on different days.

In [7]:
## 1st time
products = open('product_link.txt')
product_links = []
for item in products:
    link = item.rstrip()
    product_links.append(link)
    
frame1 = []
i = 0
for link in product_links[:1000]: ## replace 5 with 500
    print(i, link)
    product_data = get_data(link)
    if product_data:
        df = pd.DataFrame(product_data,index=[i])
        frame1.append(df)
        i+=1
        time.sleep(np.random.random()*3)

result1 = pd.concat(frame1)
result1

0 https://www.sephora.com/product/blu-mediterraneo-minature-set-P443401?icid2=products
1 https://www.sephora.com/product/colonia-P163604?icid2=products
2 https://www.sephora.com/product/rosa-nobile-P388670?icid2=products
3 https://www.sephora.com/product/blu-mediterraneo-mirto-di-panarea-P307804?icid2=products
4 https://www.sephora.com/product/blu-mediterraneo-fico-di-amalfi-P307801?icid2=products
5 https://www.sephora.com/product/colonia-miniature-set-P443400?icid2=products
6 https://www.sephora.com/product/colonia-essenza-P269110?icid2=products
7 https://www.sephora.com/product/le-nobili-collection-set-P431439?icid2=products
8 https://www.sephora.com/product/blu-mediterraneo-fico-di-amalfi-gift-set-P450815?icid2=products
9 https://www.sephora.com/product/blu-mediterraneo-arancia-di-capri-gift-set-P450817?icid2=products
10 https://www.sephora.com/product/colonia-pura-P422289?icid2=products
11 https://www.sephora.com/product/blu-mediterraneo-mandorlo-di-sicilia-P307803?icid2=products
1

93 https://www.sephora.com/product/soft-glam-eyeshadow-palette-P58930227?icid2=products
94 https://www.sephora.com/product/brow-gel-P69304?icid2=products
95 https://www.sephora.com/product/undressed-lip-set-P450064?icid2=products
96 https://www.sephora.com/product/mini-loose-highlighter-set-P451598?icid2=products
97 https://www.sephora.com/product/norvina-pro-pigment-palette-vol-3-P450846?icid2=products
98 https://www.sephora.com/product/jackie-aina-eyeshadow-palette-P449147?icid2=products
99 https://www.sephora.com/product/norvina-pro-pigment-vol-1-P448161?icid2=products
100 https://www.sephora.com/product/invigorating-rush-hair-body-wash-P376886?icid2=products
101 https://www.sephora.com/product/ingrown-hair-treatment-P170545?icid2=products
102 https://www.sephora.com/product/glycolic-exfoliating-resurfacing-wipes-P397373?icid2=products
103 https://www.sephora.com/product/facial-scrub-P9036?icid2=products
104 https://www.sephora.com/product/glycolic-facial-cleanser-P9038?icid2=produc

187 https://www.sephora.com/product/barepro-performance-wear-powder-foundation-P410543?icid2=products
188 https://www.sephora.com/product/bareminerals-matte-foundation-broad-spectrum-spf-15-P240604?icid2=products
189 https://www.sephora.com/product/mineral-veil-P61004?icid2=products
190 https://www.sephora.com/product/mineral-veil-deluxe-collector-s-edition-P450067?icid2=products
191 https://www.sephora.com/product/barepro-performance-wear-liquid-foundation-broad-spectrum-spf-20-P421001?icid2=products
192 https://www.sephora.com/product/barepro-16-hr-full-coverage-concealer-P428606?icid2=products
193 https://www.sephora.com/product/bareskin-reg-complete-coverage-serum-concealer-P399811?icid2=products
194 https://www.sephora.com/product/bareminerals-all-over-face-color-P61005?icid2=products
195 https://www.sephora.com/product/complexion-rescue-hydrating-foundation-stick-broad-spectrum-spf-25-P441829?icid2=products
196 https://www.sephora.com/product/bareminerals-nothing-beats-original-4

278 https://www.sephora.com/product/boudoir-short-hair-contour-brush-P396821?icid2=products
279 https://www.sephora.com/product/hair-redemption-butter-masque-P451206?icid2=products
280 https://www.sephora.com/product/lift-me-up-hair-thickener-P451204?icid2=products
281 https://www.sephora.com/product/volume-restore-kit-P451200?icid2=products
282 https://www.sephora.com/product/wake-up-call-volumizing-shampoo-P451210?icid2=products
283 https://www.sephora.com/product/wake-up-call-volumizing-conditioner-P451209?icid2=products
284 https://www.sephora.com/product/second-chance-repairing-shampoo-P451208?icid2=products
285 https://www.sephora.com/product/second-chance-repairing-conditioner-P451207?icid2=products
286 https://www.sephora.com/product/scalp-detox-hair-volumizing-kit-P451199?icid2=products
287 https://www.sephora.com/product/no-remorse-heat-protection-taming-spray-P451203?icid2=products
288 https://www.sephora.com/product/silver-lining-purple-butter-masque-P451205?icid2=products


369 https://www.sephora.com/product/scalp-revival-tm-scalp-soothing-travel-kit-P440959?icid2=products
370 https://www.sephora.com/product/hairdresser-s-invisible-oil-primer-P386464?icid2=products
371 https://www.sephora.com/product/refresh-restart-set-P450070?icid2=products
372 https://www.sephora.com/product/reign-dear-texture-set-P449780?icid2=products
373 https://www.sephora.com/product/hairdresser-s-invisible-oil-conditioner-P386462?icid2=products
374 https://www.sephora.com/product/thickening-volume-shampoo-P280566?icid2=products
375 https://www.sephora.com/product/hairdresser-s-invisible-oil-shampoo-P386463?icid2=products
376 https://www.sephora.com/product/thickening-dryspun-finish-P381033?icid2=products
377 https://www.sephora.com/product/hairdresser-s-invisible-oil-P375391?icid2=products
378 https://www.sephora.com/product/sleigh-belle-mega-moisture-set-P449781?icid2=products
379 https://www.sephora.com/product/thickening-volume-hairspray-P280537?icid2=products
380 https://www

469 https://www.sephora.com/product/instant-foaming-cleanser-P12978?icid2=products
470 https://www.sephora.com/product/cellu-cup-pineapple-P444217?icid2=products
471 https://www.sephora.com/product/coco-mademoiselle-P12495?icid2=products
472 https://www.sephora.com/product/chance-eau-tendre-P258612?icid2=products
473 https://www.sephora.com/product/bleu-de-chanel-eau-de-parfum-P394949?icid2=products
474 https://www.sephora.com/product/chanel-n5-P65510?icid2=products
475 https://www.sephora.com/product/chance-P257900?skuId=1001098&icid2=products
476 https://www.sephora.com/product/chance-eau-tendre-eau-de-parfum-P440093?icid2=products
477 https://www.sephora.com/product/coco-mademoiselle-intense-eau-de-parfum-P429692?icid2=products
478 https://www.sephora.com/product/gabrielle-chanel-eau-de-parfum-P422299?icid2=products
479 https://www.sephora.com/product/bleu-de-chanel-P270302?icid2=products
480 https://www.sephora.com/product/chance-P138300?icid2=products
481 https://www.sephora.com/p

,category,product_name,brand,price,Product_Id,size,item_num,love_count,rating,reviews_count,link
0,Fragrance/Value & Gift Sets/Perfume Gift Sets,Blu Mediterraneo MINIATURE Set,Acqua Di Parma,63.0,P443401,5 x 0.16oz/5mL,2218774,2594,4,4,https://www.sephora.com/product/blu-mediterran...
1,Fragrance/Men/Cologne,Colonia,Acqua Di Parma,63.0,P163604,0.7 oz/ 20 mL,2044816,2528,4.3067,75,https://www.sephora.com/product/colonia-P16360...
2,Fragrance/Women/Perfume,Rosa Nobile,Acqua Di Parma,68.0,P388670,3.4 oz/ 101 mL,1638832,4574,4.5652,69,https://www.sephora.com/product/rosa-nobile-P3...
3,Fragrance/Women/Perfume,Mirto di Panarea,Acqua Di Parma,64.0,P307804,2.5 oz/ 74 mL,1417617,2621,4.6364,22,https://www.sephora.com/product/blu-mediterran...
4,Fragrance/Women/Perfume,Fico di Amalfi,Acqua Di Parma,64.0,P307801,5 oz/ 148 mL,1417609,2466,4.5714,70,https://www.sephora.com/product/blu-mediterran...
...,...,...,...,...,...,...,...,...,...,...,...
494,Makeup/Lip/Lipstick,Superstar Lips Lipstick,Charlotte Tilbury,34.0,P446610,- nude pink,2244606,19761,3.9457,92,https://www.sephora.com/product/superstar-lips...
495,Fragrance/Women/Perfume,Chloé Eau de Parfum,Chloé,32.0,P256308,2.5 oz/ 75 mL,1239813,85350,4.5898,1753,https://www.sephora.com/product/chloe-eau-de-p...
496,Fragrance/Women/Perfume,Nomade Eau de Parfum,Chloé,32.0,P428451,2.5 oz/ 75 mL,2037471,13507,4.0462,238,https://www.sephora.com/product/nomade-eau-de-...
497,Fragrance/Women/Perfume,Roses De Chloé,Chloé,30.0,P384710,2.5 oz/ 75 mL,1580448,27662,4.5086,464,https://www.sephora.com/product/roses-de-chloe...


In [8]:
result1.to_csv('result1.csv', index=False)

In [9]:
## 2nd time
frame2 = []
i = 1000 ## replace 5 with 5000
for link in product_links[1000:2000]: ## replace [5:10] with [5000:]
    print(i, link)
    product_data = get_data(link)
    if product_data:
        df = pd.DataFrame(product_data,index=[i])
        frame2.append(df)
        i+=1
        time.sleep(np.random.random()*3)
result2 = pd.concat(frame2)
result2

500 https://www.sephora.com/product/chloe-eau-de-parfum-gift-set-P449748?icid2=products
501 https://www.sephora.com/product/chloe-chloe-eau-de-parfum-nomade-eau-de-parfum-mini-set-P452373?icid2=products
502 https://www.sephora.com/product/chlo-rollerball-P377459?icid2=products
503 https://www.sephora.com/product/chloe-l-eau-P449749?icid2=products
504 https://www.sephora.com/product/fleur-de-parfum-P412367?icid2=products
505 https://www.sephora.com/product/nomade-eau-de-parfum-travel-spray-P428450?icid2=products
506 https://www.sephora.com/product/roses-de-chloe-rollerball-P400324?icid2=products
507 https://www.sephora.com/product/chloe-nomade-eau-de-toilette-P441205?icid2=products
508 https://www.sephora.com/product/velvet-matte-lip-colour-P400560?icid2=products
509 https://www.sephora.com/product/silky-satin-lip-colour-P400556?icid2=products
510 https://www.sephora.com/product/sheer-voile-lip-colour-P400558?skuId=1750074&icid2=products
511 https://www.sephora.com/product/loubilac-lip-

591 https://www.sephora.com/product/dramatically-different-moisturizing-lotion-P381030?icid2=products
592 https://www.sephora.com/product/from-party-to-pjs-P449616?icid2=products
593 https://www.sephora.com/product/even-better-makeup-spf-15-P234967?icid2=products
594 https://www.sephora.com/product/dramatically-different-moisturizing-gel-P122900?icid2=products
595 https://www.sephora.com/product/all-about-eyes-rich-P174502?icid2=products
596 https://www.sephora.com/product/take-the-day-off-makeup-remover-for-lids-lashes-lips-P122735?icid2=products
597 https://www.sephora.com/product/beyond-perfecting-foundation-concealer-P393325?icid2=products
598 https://www.sephora.com/product/stay-matte-sheer-pressed-powder-P122748?icid2=products
599 https://www.sephora.com/product/take-the-day-off-cleansing-balm-P126301?icid2=products
600 https://www.sephora.com/product/high-impact-mascara-P122912?icid2=products
601 https://www.sephora.com/product/girls-night-out-collection-P432412?icid2=products
6

688 https://www.sephora.com/product/active-moist-P423142?icid2=products
689 https://www.sephora.com/product/dynamic-skin-recovery-spf50-P443354?icid2=products
690 https://www.sephora.com/product/multivitamin-power-recovery-masque-P423130?icid2=products
691 https://www.sephora.com/product/age-smart-phyto-nature-firming-serum-P449177?icid2=products
692 https://www.sephora.com/product/biolumin-c-serum-P443370?icid2=products
693 https://www.sephora.com/product/intensive-moisture-balance-P443358?icid2=products
694 https://www.sephora.com/product/super-rich-repair-P423135?icid2=products
695 https://www.sephora.com/product/ultra-defining-gel-strong-hold-no-crunch-styler-P378332?icid2=products
696 https://www.sephora.com/product/devadryer-devafuser-dryer-diffuser-combo-for-all-curl-kind-P398937?icid2=products
697 https://www.sephora.com/product/supercream-coconut-curl-styler-P399687?icid2=products
698 https://www.sephora.com/product/one-condition-P378326?icid2=products
699 https://www.sephora.

782 https://www.sephora.com/product/dr-barbara-sturm-calming-serum-P447788?icid2=products
783 https://www.sephora.com/product/dr-barbara-sturm-face-cream-light-P449601?icid2=products
784 https://www.sephora.com/product/dr-barbara-sturm-flight-essentials-kit-P449603?icid2=products
785 https://www.sephora.com/product/pores-no-more-pore-refiner-primer-P139005?icid2=products
786 https://www.sephora.com/product/microdermabrasion-age-defying-exfoliator-P45503?icid2=products
787 https://www.sephora.com/product/no-more-baggage-eye-de-puffing-gel-P421759?icid2=products
788 https://www.sephora.com/product/needles-no-more-neck-sculpting-cream-P440302?icid2=products
789 https://www.sephora.com/product/do-not-age-with-dr-brandt-triple-peptide-eye-cream-P383053?icid2=products
790 https://www.sephora.com/product/needles-no-more-P391802?icid2=products
791 https://www.sephora.com/product/do-not-age-with-dr-brandt-moisturizing-neck-cream-P383052?icid2=products
792 https://www.sephora.com/product/pores-n

AttributeError: 'NoneType' object has no attribute 'contents'

In [11]:
result2 = pd.concat(frame2)
result2.to_csv('result2.csv', index=False)
result2

,category,product_name,brand,price,Product_Id,size,item_num,love_count,rating,reviews_count,link
500,Fragrance/Value & Gift Sets/Perfume Gift Sets,Chloé Eau de Parfum Gift Set,Chloé,115.0,P449748,na,2253912,774,na,na,https://www.sephora.com/product/chloe-eau-de-p...
501,Fragrance/Value & Gift Sets/Perfume Gift Sets,Chloé Eau de Parfum & Nomade Eau de Parfum Min...,Chloé,29.0,P452373,na,2249258,3004,2.8333,6,https://www.sephora.com/product/chloe-chloe-ea...
502,Fragrance/Women/Rollerballs & Travel Size,Chloé Rollerball,Chloé,32.0,P377459,.33oz /10ml,1284025,0,4.6047,43,https://www.sephora.com/product/chlo-rollerbal...
503,Fragrance/Women/Perfume,Chloe L'Eau,Chloé,68.0,P449749,3.4 oz/ 100 mL,2249241,1027,4.0949,316,https://www.sephora.com/product/chloe-l-eau-P4...
504,Fragrance/Women/Perfume,Fleur de Parfum,Chloé,30.0,P412367,2.5 oz/ 74 mL,1854595,6353,4.1803,61,https://www.sephora.com/product/fleur-de-parfu...
...,...,...,...,...,...,...,...,...,...,...,...
812,Skincare/Value & Gift Sets,Cera-Care Experiment Kit,Dr. Jart+,40.0,P449891,na,2264158,4813,4.2857,7,https://www.sephora.com/product/cera-care-expe...
813,Skincare/Moisturizers/Moisturizers,Ceramidin™ Cream,Dr. Jart+,48.0,P434363,1.69 oz/ 50 mL,2077840,31240,4.5984,1108,https://www.sephora.com/product/ceramidin-tm-c...
814,Makeup/Face/BB & CC Cream,Premium Beauty Balm SPF 45,Dr. Jart+,39.0,P285403,1.5 oz/ 40 mL,1327915,56203,4.2292,2806,https://www.sephora.com/product/premium-beauty...
815,Skincare/Value & Gift Sets,Mini Must-Haves Experiment Kit,Dr. Jart+,20.0,P449890,na,2264166,6870,4.5,8,https://www.sephora.com/product/mini-must-have...


In [12]:
result = pd.concat([result1,result2])
result

,category,product_name,brand,price,Product_Id,size,item_num,love_count,rating,reviews_count,link
0,Fragrance/Value & Gift Sets/Perfume Gift Sets,Blu Mediterraneo MINIATURE Set,Acqua Di Parma,63.0,P443401,5 x 0.16oz/5mL,2218774,2594,4,4,https://www.sephora.com/product/blu-mediterran...
1,Fragrance/Men/Cologne,Colonia,Acqua Di Parma,63.0,P163604,0.7 oz/ 20 mL,2044816,2528,4.3067,75,https://www.sephora.com/product/colonia-P16360...
2,Fragrance/Women/Perfume,Rosa Nobile,Acqua Di Parma,68.0,P388670,3.4 oz/ 101 mL,1638832,4574,4.5652,69,https://www.sephora.com/product/rosa-nobile-P3...
3,Fragrance/Women/Perfume,Mirto di Panarea,Acqua Di Parma,64.0,P307804,2.5 oz/ 74 mL,1417617,2621,4.6364,22,https://www.sephora.com/product/blu-mediterran...
4,Fragrance/Women/Perfume,Fico di Amalfi,Acqua Di Parma,64.0,P307801,5 oz/ 148 mL,1417609,2466,4.5714,70,https://www.sephora.com/product/blu-mediterran...
...,...,...,...,...,...,...,...,...,...,...,...
812,Skincare/Value & Gift Sets,Cera-Care Experiment Kit,Dr. Jart+,40.0,P449891,na,2264158,4813,4.2857,7,https://www.sephora.com/product/cera-care-expe...
813,Skincare/Moisturizers/Moisturizers,Ceramidin™ Cream,Dr. Jart+,48.0,P434363,1.69 oz/ 50 mL,2077840,31240,4.5984,1108,https://www.sephora.com/product/ceramidin-tm-c...
814,Makeup/Face/BB & CC Cream,Premium Beauty Balm SPF 45,Dr. Jart+,39.0,P285403,1.5 oz/ 40 mL,1327915,56203,4.2292,2806,https://www.sephora.com/product/premium-beauty...
815,Skincare/Value & Gift Sets,Mini Must-Haves Experiment Kit,Dr. Jart+,20.0,P449890,na,2264166,6870,4.5,8,https://www.sephora.com/product/mini-must-have...


In [13]:
## Save the result as csv. 
result.to_csv('result.csv', index=False)

In [38]:
# 3rd time
frame3 = []
i = 2899  # replace 5 with 5000
for link in product_links[2899:]:  # replace [5:10] with [5000:]
    print(i, link)
    product_data = get_data(link)
    if product_data:
        df = pd.DataFrame(product_data, index=[i])
        frame3.append(df)
        i += 1
        time.sleep(np.random.random()*3)
result3 = pd.concat(frame3)
result3

2899 https://www.sephora.com/product/born-in-roma-P449116?icid2=products
2900 https://www.sephora.com/product/valentino-donna-born-in-roma-eau-de-parfum-set-P452822?icid2=products
2901 https://www.sephora.com/product/born-in-roma-travel-spray-P449117?icid2=products
2902 https://www.sephora.com/product/effortless-lash-collection-P429277?icid2=products
2903 https://www.sephora.com/product/the-effortless-kit-P429278?icid2=products
2904 https://www.sephora.com/product/naughty-but-nice-travel-case-P450061?icid2=products
2905 https://www.sephora.com/product/fluff-n-glam-collection-P449091?icid2=products
2906 https://www.sephora.com/product/minimalist-mink-lash-collection-P447558?icid2=products
2907 https://www.sephora.com/product/too-easy-lash-applicator-P420514?icid2=products
2908 https://www.sephora.com/product/lash-adhesive-P420513?icid2=products
2909 https://www.sephora.com/product/silk-lash-collection-P420510?icid2=products
2910 https://www.sephora.com/product/luminous-lash-collection-P

2994 https://www.sephora.com/product/quiet-time-diffuser-set-P443562?icid2=products
2995 https://www.sephora.com/product/organic-lavender-oil-P433637?icid2=products
2996 https://www.sephora.com/product/organic-eucalyptus-essential-oil-P433633?icid2=products
2997 https://www.sephora.com/product/organic-sweet-orange-essential-oil-P433634?icid2=products
2998 https://www.sephora.com/product/organic-peppermint-essential-oil-P433632?icid2=products
2999 https://www.sephora.com/product/frankincense-essential-oil-P433631?icid2=products
3000 https://www.sephora.com/product/sleep-duo-essential-oil-set-P444045?icid2=products
3001 https://www.sephora.com/product/vitruvi-pacific-blend-P448179?icid2=products
3002 https://www.sephora.com/product/pink-grapefruit-essential-oil-P433635?icid2=products
3003 https://www.sephora.com/product/geranium-essential-oil-P433636?icid2=products
3004 https://www.sephora.com/product/vitruvi-dusk-blend-P448178?icid2=products
3005 https://www.sephora.com/product/celery-g

3082 https://www.sephora.com/product/vetiver-elemi-eau-de-parfum-P428715?icid2=products
3083 https://www.sephora.com/product/grapefruit-lime-eau-de-parfum-P428702?icid2=products
3084 https://www.sephora.com/product/vetiver-elemi-eau-de-parfum-rollerball-P428716?icid2=products
3085 https://www.sephora.com/product/jasmine-neroli-eau-de-parfum-rollerball-P428705?icid2=products
3086 https://www.sephora.com/product/orange-blossom-eau-de-parfum-rollerball-P428707?icid2=products
3087 https://www.sephora.com/product/8greens-8greens-gummies-dietary-supplement-P452918?icid2=products
3088 https://www.sephora.com/product/8greens-dietary-supplement-P437988?icid2=products


,category,product_name,brand,price,Product_Id,size,item_num,love_count,rating,reviews_count,link
2899,Fragrance/Women/Perfume,Donna Born In Roma Eau de Parfum,Valentino,30.0,P449116,1.7 oz/ 50 mL,2249688,6417,4.7492,299,https://www.sephora.com/product/born-in-roma-P...
2900,Fragrance/Value & Gift Sets/Perfume Gift Sets,Donna Born In Roma Eau de Parfum Set,Valentino,115.0,P452822,1.7 oz/ 50 mL & 3.4 oz/ 100 mL,2249662,902,5,4,https://www.sephora.com/product/valentino-donn...
2901,Fragrance/Women/Rollerballs & Travel Size,Donna Born In Roma Eau de Parfum Travel Spray,Valentino,30.0,P449117,0.33 oz/ 10 mL,2249712,2398,3.9,20,https://www.sephora.com/product/born-in-roma-t...
2902,Makeup/Eye/False Eyelashes,Effortless - No Trim - Natural Lash Collection,Velour Lashes,26.0,P429277,- gradient flare and natural volume on the ou...,2203487,33667,4.4738,821,https://www.sephora.com/product/effortless-las...
2903,Makeup/Eye/False Eyelashes,Effortless Kit - No Trim No Measure Natural La...,Velour Lashes,29.0,P429278,- naturally flared & crisscrossed,2044006,28337,4.0333,210,https://www.sephora.com/product/the-effortless...
...,...,...,...,...,...,...,...,...,...,...,...
3084,Fragrance/Women/Rollerballs & Travel Size,Vetiver Elemi Eau de Parfum Rollerball,The 7 Virtues,29.0,P428716,0.33 oz/ 11 mL,2047702,1230,4.6,15,https://www.sephora.com/product/vetiver-elemi-...
3085,Fragrance/Women/Rollerballs & Travel Size,Jasmine Neroli Eau de Parfum Rollerball,The 7 Virtues,29.0,P428705,0.33 oz/ 11 mL,2047678,1880,4.3226,31,https://www.sephora.com/product/jasmine-neroli...
3086,Fragrance/Women/Rollerballs & Travel Size,Orange Blossom Eau de Parfum Rollerball,The 7 Virtues,29.0,P428707,0.33 oz/ 11 mL,2047728,2009,4.2083,24,https://www.sephora.com/product/orange-blossom...
3087,Skincare/Wellness/Beauty Supplements,8Greens Gummies Dietary Supplement,8Greens,45.0,P452918,60 Gummies,2298891,1668,4.3333,9,https://www.sephora.com/product/8greens-8green...


In [39]:
result3 = pd.concat(frame3)
result3

,category,product_name,brand,price,Product_Id,size,item_num,love_count,rating,reviews_count,link
2899,Fragrance/Women/Perfume,Donna Born In Roma Eau de Parfum,Valentino,30.0,P449116,1.7 oz/ 50 mL,2249688,6417,4.7492,299,https://www.sephora.com/product/born-in-roma-P...
2900,Fragrance/Value & Gift Sets/Perfume Gift Sets,Donna Born In Roma Eau de Parfum Set,Valentino,115.0,P452822,1.7 oz/ 50 mL & 3.4 oz/ 100 mL,2249662,902,5,4,https://www.sephora.com/product/valentino-donn...
2901,Fragrance/Women/Rollerballs & Travel Size,Donna Born In Roma Eau de Parfum Travel Spray,Valentino,30.0,P449117,0.33 oz/ 10 mL,2249712,2398,3.9,20,https://www.sephora.com/product/born-in-roma-t...
2902,Makeup/Eye/False Eyelashes,Effortless - No Trim - Natural Lash Collection,Velour Lashes,26.0,P429277,- gradient flare and natural volume on the ou...,2203487,33667,4.4738,821,https://www.sephora.com/product/effortless-las...
2903,Makeup/Eye/False Eyelashes,Effortless Kit - No Trim No Measure Natural La...,Velour Lashes,29.0,P429278,- naturally flared & crisscrossed,2044006,28337,4.0333,210,https://www.sephora.com/product/the-effortless...
...,...,...,...,...,...,...,...,...,...,...,...
3084,Fragrance/Women/Rollerballs & Travel Size,Vetiver Elemi Eau de Parfum Rollerball,The 7 Virtues,29.0,P428716,0.33 oz/ 11 mL,2047702,1230,4.6,15,https://www.sephora.com/product/vetiver-elemi-...
3085,Fragrance/Women/Rollerballs & Travel Size,Jasmine Neroli Eau de Parfum Rollerball,The 7 Virtues,29.0,P428705,0.33 oz/ 11 mL,2047678,1880,4.3226,31,https://www.sephora.com/product/jasmine-neroli...
3086,Fragrance/Women/Rollerballs & Travel Size,Orange Blossom Eau de Parfum Rollerball,The 7 Virtues,29.0,P428707,0.33 oz/ 11 mL,2047728,2009,4.2083,24,https://www.sephora.com/product/orange-blossom...
3087,Skincare/Wellness/Beauty Supplements,8Greens Gummies Dietary Supplement,8Greens,45.0,P452918,60 Gummies,2298891,1668,4.3333,9,https://www.sephora.com/product/8greens-8green...


In [31]:
result3.to_csv('result3.csv', index=False)

In [40]:
result = pd.concat([result,result3])
result

,category,product_name,brand,price,Product_Id,size,item_num,love_count,rating,reviews_count,link
0,Fragrance/Value & Gift Sets/Perfume Gift Sets,Blu Mediterraneo MINIATURE Set,Acqua Di Parma,63.0,P443401,5 x 0.16oz/5mL,2218774,2594,4,4,https://www.sephora.com/product/blu-mediterran...
1,Fragrance/Men/Cologne,Colonia,Acqua Di Parma,63.0,P163604,0.7 oz/ 20 mL,2044816,2528,4.3067,75,https://www.sephora.com/product/colonia-P16360...
2,Fragrance/Women/Perfume,Rosa Nobile,Acqua Di Parma,68.0,P388670,3.4 oz/ 101 mL,1638832,4574,4.5652,69,https://www.sephora.com/product/rosa-nobile-P3...
3,Fragrance/Women/Perfume,Mirto di Panarea,Acqua Di Parma,64.0,P307804,2.5 oz/ 74 mL,1417617,2621,4.6364,22,https://www.sephora.com/product/blu-mediterran...
4,Fragrance/Women/Perfume,Fico di Amalfi,Acqua Di Parma,64.0,P307801,5 oz/ 148 mL,1417609,2466,4.5714,70,https://www.sephora.com/product/blu-mediterran...
...,...,...,...,...,...,...,...,...,...,...,...
3084,Fragrance/Women/Rollerballs & Travel Size,Vetiver Elemi Eau de Parfum Rollerball,The 7 Virtues,29.0,P428716,0.33 oz/ 11 mL,2047702,1230,4.6,15,https://www.sephora.com/product/vetiver-elemi-...
3085,Fragrance/Women/Rollerballs & Travel Size,Jasmine Neroli Eau de Parfum Rollerball,The 7 Virtues,29.0,P428705,0.33 oz/ 11 mL,2047678,1880,4.3226,31,https://www.sephora.com/product/jasmine-neroli...
3086,Fragrance/Women/Rollerballs & Travel Size,Orange Blossom Eau de Parfum Rollerball,The 7 Virtues,29.0,P428707,0.33 oz/ 11 mL,2047728,2009,4.2083,24,https://www.sephora.com/product/orange-blossom...
3087,Skincare/Wellness/Beauty Supplements,8Greens Gummies Dietary Supplement,8Greens,45.0,P452918,60 Gummies,2298891,1668,4.3333,9,https://www.sephora.com/product/8greens-8green...


In [41]:
result.to_csv('result.csv', index=False)

In [ ]:
# Old version get_data function

def get_data(link):
    wb_data = requests.get(link)
    soup = bs4.BeautifulSoup(wb_data.text,'lxml')
    
    ## para1 is to get info of sort,price,product name and brand
    para1 = soup.find_all(attrs={"type": "application/ld+json"})
    sort = ''
    for content1 in para1[:2]:
        content1 = content1.string
        try:
            content1_dict = json.loads(content1)
        except:
            continue
        try:
            content1_lst = content1_dict['itemListElement']
            for item in content1_lst:
                sort= sort+item['item']['name']+ '/'
        except:
            price = content1_dict['offers'][0]['price'] 
            product_name = content1_dict['name']          
            brand = content1_dict['brand']
    
    ## para2 is to get info of size and item number
    para2 = soup.find(attrs={"class": "css-1qf1va5"})
    para2 = str(para2)
    try:
        size = re.findall(R'SIZE.+?\>(.*?)\<span',para2)[0]
    except:
        size ='na'
    try:
        item_num = re.findall(R'ITEM.+?\>(.*?)\</div\>',para2)[0]
    except:
        item_num = 'na'
    
    ## para3 is to get info of love counts
    para3 = soup.find(attrs={"data-at": "product_love_count"})
    para3 = str(para3)
    try:
        love_count = re.findall(R'\>([0-9].+)\</span\>',para3)[0]
    except:
        love_count = 'na'
    
    ## para4 is to get info of rating and review counts
    para4 = soup.find(attrs={"id": "linkJSON"})
    para4 = str(para4)
    ratings = re.findall(R'\"rating\"\:(.*?)\,',para4)
    reviews = re.findall(R'\"reviews\"\:(.*?)\,',para4)
    
    
    try:
        rating = ratings[0]
    except:
        rating = 'na'
    
    try:
        reviews_count = reviews[0]
    except:
        reviews_count = 'na'
    
    if size == []:
            size = 'na'
    if item_num ==[]:
        item_num = 'na'
    if love_count == []:
        love_count = 'na'
     
    dic1 = {}
    dic1['category'] = sort
    dic1['product_name']= product_name
    dic1['brand'] = brand
    dic1['price'] = price
    dic1['Product_Id'] = re.findall(R'P[0-9]{3,6}',link)
    dic1['size'] = size
    dic1['item_num'] = item_num
    dic1['love_count'] = love_count
    dic1['rating'] = rating
    dic1['reviews_count'] = reviews_count
    dic1['link'] = link
    
    return dic1